In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interactive, interact, FloatSlider, IntRangeSlider

from dask.distributed import Client, progress, wait
import numpy as np
import dask.array as da
import dask
from numcodecs import Blosc

import sys

sys.path.insert(0, '../thirdparty/singleTrace_d2geo/')
import CompleTrace as cmpAttr

# cd ../thirdparty/Semblance
# python setup.py install
from semblance import semblance

/glb/data/cdis_projects/users/usnrej/attEngine/python-env/lib/python3.5/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
import attengineshell as atte

In [3]:
client = Client(address='tcp://10.53.2.149:8786',  # Highlight this!!
                set_as_default=True, name='Client_1', 
                direct_to_workers=True)

In [4]:
# inp_file = '/glb/data/cdis_projects/users/usnrej/attEngine/data/mars_obn2007.npy'
# inp_file = '/glb/data/cdis_projects/users/usnrej/attEngine/data/ang10.npy'
# inp_file_zarr = '/glb/data/cdis_projects/users/usnrej/attEngine/data/ang10.zarr'
# out_file = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/ang10_out.npy'
# out_file_zarr = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/ang10_out.zarr'

inp_file = '/glb/data/cdis_projects/users/usnrej/attEngine/data/fourpoint/seis_fourpoint.npy'
inp_file_zarr = '/glb/data/cdis_projects/users/usnrej/attEngine/data/fourpoint/seis_fourpoint.zarr'
out_file_semblance = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/seis_fourpoint_out_sem.npy'
out_file_semblance_zarr = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/seis_fourpoint_out_sem.zarr'
out_file_envelope = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/seis_fourpoint_out_envelope.npy'
out_file_envelope_zarr = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/seis_fourpoint_out_envelope._zarr'
out_file_instapha = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/seis_fourpoint_out_insta_phase.npy'
out_file_instapha_zarr = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/seis_fourpoint_out_insta_phase.zarr'
out_file_zarr = '/glb/data/cdis_projects/users/usnrej/attEngine/dataOut/seis_fourpoint_out.zarr'

In [5]:
# ate.ATScheduler
# client.restart()
semblance = atte.Semblance()
semblance.set_params(client, inp_file_zarr)
data_in, res_semblance  = semblance.run()

In [6]:
# res_semblance = res_semblance.to_zarr(out_file_semblance_zarr, compute=True, overwrite=True, return_stored=True)
# x = x.store(return_stored=True)
# res_semblance.to_zarr(out_file_semblance_zarr, overwrite=True, compressor=Blosc(cname='zstd', clevel=1, shuffle=Blosc.SHUFFLE))

In [7]:
# # client.restart()
# envelope = atte.SingleTrace.Envelope()
# # envelope.set_params(client, inp_file_zarr)
# envelope.set_params(client, data_in)
# data_in, res_envel = envelope.run_new()

In [8]:
# # client.restart()
# instaphase = atte.SingleTrace.InstantaneousPhase()
# # instaphase.set_params(client, inp_file_zarr)
# instaphase.set_params(client, data_in)
# data_in, res_instaphase = instaphase.run_new()

In [7]:
# # Saving using different compression algorithms (which one releases the GIL properly?)

# res_semblance.to_zarr(out_file_semblance_zarr, compute=True, overwrite=True, 
# #                       compressor=Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0))
# #                       compressor=Blosc(cname='zstd', clevel=1, shuffle=Blosc.SHUFFLE))
#                       compressor=Blosc(cname='zstd', clevel=1, shuffle=Blosc.SHUFFLE))  
# res_envel.to_zarr(out_file_envelope_zarr, compute=True, overwrite=True,
#                   compressor=Blosc(cname='zstd', clevel=1, shuffle=Blosc.SHUFFLE))
# res_instaphase.to_zarr(out_file_instapha_zarr, compute=True, overwrite=True,
#                        compressor=Blosc(cname='zstd', clevel=1, shuffle=Blosc.SHUFFLE))

res_semblance.to_zarr(out_file_semblance_zarr, compute=True, overwrite=True, compressor=None)
# res_envel.to_zarr(out_file_envelope_zarr, compute=True, overwrite=True, compressor=None)
# res_instaphase.to_zarr(out_file_instapha_zarr, compute=True, overwrite=True, compressor=None)

In [10]:
# res_instaphase = res_instaphase.to_zarr(out_file_semblance_zarr, compute=True, overwrite=True, return_stored=True)


In [6]:
# fig=plt.figure(figsize=(16, 12), dpi= 80, facecolor='w', edgecolor='k')
plt.rcParams['figure.figsize'] = [16, 12]
def f(slice, xran, zran):
    xini = xran[0]
    xfin = xran[1]
    zini = zran[0]
    zfin = zran[1]
#     print(xini)
    if xfin <= xini or zfin <= zini:
        return
    plt.subplot(211)
#     data = 
    atte.plot_result_v01(np.rot90(data_in[xini:xfin,slice,zini:zfin],1))
    plt.title('Input data')
    plt.subplot(212)
    atte.plot_result_v01(np.rot90(res_semblance[xini:xfin,slice,zini:zfin],1))
    plt.title('Semblance')
#     plt.subplot(413)
#     atte.plot_result_v01(np.rot90(res_envel[xini:xfin,slice,zini:zfin],1))
#     plt.title('Envelope')
#     plt.subplot(414)
#     atte.plot_result_v01(np.rot90(res_instaphase[xini:xfin,slice,zini:zfin],1))
#     plt.title('Input data')
    
xmax, slicemax, zmax = data_in.shape

interact(f,slice=FloatSlider(value=812, min=0, max=slicemax-1, step=1, continuous_update=False),
         xran=IntRangeSlider(value=[675, 1395], min=0, max=xmax-1, step=1, continuous_update=False),
         zran=IntRangeSlider(value=[210, 829], min=0, max=zmax-1, step=1, continuous_update=False)
        )

interactive(children=(FloatSlider(value=812.0, continuous_update=False, description='slice', max=1956.0, step=…

<function __main__.f(slice, xran, zran)>

In [ ]:
# fig=plt.figure(figsize=(16, 12), dpi= 80, facecolor='w', edgecolor='k')
plt.rcParams['figure.figsize'] = [16, 12]
def f(sran, xran, zran):
    sini = sran[0]
    sfin = sran[1]
#     xini = xran[0]
#     xfin = xran[1]
    zini = zran[0]
    zfin = zran[1]
#     print(xini)
#     if xfin <= xini or zfin <= zini:
#         return
    plt.subplot(411)
    atte.plot_result_v01(np.rot90(data_in[xran, sini:sfin,zini:zfin],1))
    plt.title('Input data')
    plt.subplot(412)
    atte.plot_result_v01(np.rot90(res_semblance[xran, sini:sfin,zini:zfin],1))
    plt.title('Semblance')
    plt.subplot(413)
    atte.plot_result_v01(np.rot90(res_envel[xran, sini:sfin,zini:zfin],1))
    plt.title('Envelope')
    plt.subplot(414)
    atte.plot_result_v01(np.rot90(res_instaphase[xran, sini:sfin,zini:zfin],1))
    plt.title('Input data')
    
xmax, slicemax, zmax = data_in.shape

interact(f,
         sran=IntRangeSlider(value=[700, 1611], min=0, max=slicemax-1, step=1, continuous_update=False),
         xran=FloatSlider(value=675, min=0, max=xmax-1, step=1, continuous_update=False),
         zran=IntRangeSlider(value=[210, 829], min=0, max=zmax-1, step=1, continuous_update=False)
        )

In [9]:
# fig=plt.figure(figsize=(16, 12), dpi= 80, facecolor='w', edgecolor='k')
plt.rcParams['figure.figsize'] = [16, 12]
def f(sran, xran, zran):
    sini = sran[0]
    sfin = sran[1]
    xini = xran[0]
    xfin = xran[1]
#     zini = zran[0]
#     zfin = zran[1]
#     print(xini)
#     if xfin <= xini or zfin <= zini:
#         return
    plt.subplot(411)
    atte.plot_result_v01(np.rot90(data_in[xini:xfin, sini:sfin,zran],1))
    plt.title('Input data')
    plt.subplot(412)
    atte.plot_result_v01(np.rot90(res_semblance[xini:xfin, sini:sfin,zran],1))
    plt.title('Semblance')
    plt.subplot(413)
    atte.plot_result_v01(np.rot90(res_envel[xini:xfin, sini:sfin,zran],1))
    plt.title('Envelope')
    plt.subplot(414)
    atte.plot_result_v01(np.rot90(res_instaphase[xini:xfin, sini:sfin,zran],1))
    plt.title('Input data')
    
xmax, slicemax, zmax = data_in.shape

interact(f,
         sran=IntRangeSlider(value=[700, 1600], min=0, max=slicemax-1, step=1, continuous_update=False),
         xran=IntRangeSlider(value=[300, 1700], min=0, max=xmax-1, step=1, continuous_update=False),
         zran=FloatSlider(value=210, min=0, max=zmax-1, step=1, continuous_update=False)
        )

interactive(children=(IntRangeSlider(value=(700, 1600), continuous_update=False, description='sran', max=1956)…

<function __main__.f(sran, xran, zran)>

In [ ]:
# wx = 1
# wy = 1
# wz = 4
# def nsemblance(f):
# #     print(f.shape)
#     nx=f.shape[0]
#     ny = f.shape[1]
#     nz = f.shape[2]

# #     output  = np.zeros([nx, ny, nz])
# # #     output[0] = f.shape[0]
# #     output[1] = f.shape[1]
# # #     output[2] = f.shape[2]
# #     return output
    
#     output  = np.zeros([nx-2*wx, ny-2*wy, nz-2*wz], dtype=np.float32)
# #     output  = np.zeros([nx, ny, nz])
#     for iz in range(wz,nz-wz):
#         for iy in range(wy,ny-wy):
#             for ix in range(wx,nx-wx):

#                 xfrom = max(ix - wx, 0)
#                 xto   = min(ix + wx + 1, nx)

#                 yfrom = max(iy - wy, 0)
#                 yto   = min(iy + wy + 1, ny)

#                 zfrom = max(iz - wz, 0)
#                 zto   = min(iz + wz + 1, nz)
                
# #                 print("---")
# #                 print(ix,iy,iz)
# #                 print(xfrom,xto)
# #                 print(yfrom,yto)
# #                 print(zfrom,zto)

#                 sumSqrSum = 0.0
#                 sumSumSqr = 0.0

#                 for k in range(zfrom,zto):
#                     sum1 = 0
#                     sum2 = 0
#                     nuvalid = 0
#                     for i in range(xfrom,xto):
#                         for j in range(yfrom,yto):
#                             value = f[i,j,k]
# #                             if (value == value): 
#                             nuvalid +=1
#                             sum1 += value
#                             sum2 += value*value

#                     sumSqrSum+=sum1*sum1
#                     sumSumSqr+=sum2*nuvalid

#                 output[ix-wx,iy-wy,iz-wz] = 0 if sumSumSqr == 0 else 1 - sumSqrSum / sumSumSqr

#     return output
